<a href="https://colab.research.google.com/github/Kudston/llms_example_projects/blob/main/A_small_transformer_model_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BUILDING A SMALL TRANSFORMER MODEL FROM SCRATCH**

Update the packages

In [ ]:
import os
os.environ.setdefault("KERAS_BACKEND", "jax")

'jax'

In [ ]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras
!pip install -q --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


Import the packages

In [ ]:
import keras_nlp
import tensorflow as tf
import keras

keras.mixed_precision.set_global_policy("mixed_float16")

DOWNLOAD DATA FOR USE


In [ ]:
!curl -O "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
!tar -xf "aclImdb_v1.tar.gz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  38.0M      0  0:00:02  0:00:02 --:--:-- 38.0M


In [ ]:
rm -r "aclImdb/train/unsup"

In [ ]:
train_dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb/train", batch_size=32, validation_split=0.2, subset="training", seed=42
)
val_dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb/train", batch_size=32, validation_split=0.2, subset="validation", seed=42
)
test_dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb/test", batch_size=32
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


CREATE THE TOKENIZER

In [ ]:
##USING WORLD PIECE TOKENIZER
vocabs = keras_nlp.tokenizers.compute_word_piece_vocabulary(
  train_dataset.map(lambda x,y: x),
  vocabulary_size= 20_000,
  lowercase=True,
  strip_accents=True,
  reserved_tokens=['[PAD]','[START]','[END]','[CLS]','[SEP]','[MASK]','[UNK]']
)
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocabs,
    sequence_length=512,
    lowercase=True,
    strip_accents=True,
    oov_token='[UNK]'
)

CREATE THE TOKEN PARKER

In [ ]:
packer = keras_nlp.layers.StartEndPacker(
    sequence_length=512,
    start_value=tokenizer.token_to_id('[START]'),
    end_value=tokenizer.token_to_id('[END]'),
    pad_value=tokenizer.token_to_id('[PAD]'),
)

def preprocess(x, y):
  packed_inputs = packer(tokenizer(x))
  return packed_inputs, y



In [ ]:
##PREPROCESS THE DATASETS
pretrained_ds = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
val_ds = val_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

In [ ]:
print(pretrained_ds.unbatch().take(1).get_single_element())

(<tf.Tensor: shape=(512,), dtype=int32, numpy=
array([    1,   271,   232,   111,   375,   392,  1060,    21,   109,
         114,   165,    46,   503,   276,   102,  1134,    21,    54,
         485,    19,   185,   148,   119,   124,    46,   173,   114,
         115,   743,  6155,  1883,    21,   108,   107,    19,   682,
         124,   102,   223,   177,  1979,    21,   121,   142,   101,
        1380,   104,  2051,  1360,   153,   196,   114,   115,   123,
         573,   751,    21,   109,   114,   111,   507,     8,    54,
         314,   109,   114,  1379,   127,   685,   118,    46,   114,
        4540,   108,     9,  2854,     9,   102,   135,   107,   111,
        2995,   141,   104,   167,    19,    54,   527,    14,    65,
        5438,  3996,   107,   153,   102,  2526,   107,   444,   115,
         167,    21,   119,   148,   790,   167,    46,  4896,   848,
          19,   116,   109,   114,   139,  3026,   157,   582,    19,
         120,   104,   874,   188,   167,  

DESIGN A SMALL TRANSFORMER MODEL

In [ ]:
inputs = keras.Input(shape=(None,), dtype=tf.int32)
##Add a token position embedding
outputs = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=20000,
    sequence_length=512,
    embedding_dim=128,
)(inputs)

##Add a transformer layer
outputs = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=128,
    num_heads=2,
    dropout=0.2
)(outputs)

##Add another transformer layer
outputs = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=128,
    num_heads=2,
    dropout=0.2
)(outputs)

##Add a pooling layer
outputs = keras.layers.Dense(2, activation='softmax')(outputs[:,0,:])

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding_2       │ (None, None, 128)           │       2,625,536 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_2                │ (None, None, 128)           │          99,584 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_3                │ (None, None, 128)           │          99,584 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ get_item_1 (GetItem)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,824,962 (10.78 MB)

 Trainable params: 2,824,962 (10.78 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(3e-4),
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

In [ ]:
model.fit(
    pretrained_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - loss: 0.3750 - sparse_categorical_accuracy: 0.9360 - val_loss: 0.4354 - val_sparse_categorical_accuracy: 0.8740
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 37ms/step - loss: 0.3641 - sparse_categorical_accuracy: 0.9471 - val_loss: 0.4574 - val_sparse_categorical_accuracy: 0.8534
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - loss: 0.3681 - sparse_categorical_accuracy: 0.9439 - val_loss: 0.4349 - val_sparse_categorical_accuracy: 0.8754
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - loss: 0.3586 - sparse_categorical_accuracy: 0.9542 - val_loss: 0.4417 - val_sparse_categorical_accuracy: 0.8686
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - loss: 0.3596 - sparse_categorical_accuracy: 0.9522 - val_loss: 0.4496 - val_sparse_categorical_accuracy: 0.8610
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - loss: 0.3579 - sparse_categorical_accuracy: 0.9548 - val_loss: 0.4403 - val_sparse_categorical_accuracy: